In [16]:
# Importações
import pandas as pd
import requests as rq
import zipfile
import datetime
import io
import os

In [17]:
# Atribuições
# URL do arquivo zip que você deseja baixar

url = 'https://portaldatransparencia.gov.br/download-de-dados/emendas-parlamentares/UNICO'

headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'Referer': f"{url}"
}

data = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

name_directory = f"Emenda_Parlamentar_{data.split(' ')[0].replace('-','')}"

if not os.path.exists(name_directory):
    os.makedirs(name_directory)

response = rq.get(url, headers=headers)

In [18]:
# Extraindo arquivos do zip e inserindo no diretório desejado
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            #zip_ref.extractall(path=namedirectory)
            zip_ref.extractall(path=name_directory)

In [20]:
# Lendo arquivo CSV usando pd.read_csv
df_csv = pd.read_csv(f'{name_directory}/Emendas.csv', sep=';', encoding='ISO-8859-1')

KeyboardInterrupt: 